# Extract Data Oracle Database

In [1]:
import pandas as pd 
import sqlalchemy as sa 
from sqlalchemy import create_engine
import sqlite3
import cx_Oracle as cx 
import warnings

In [5]:
connection = cx.connect(user=, password="",
                               dsn="")

In [4]:
connection.close()

In [3]:
# Loading Data
#Stage Area
#extract data from Oracle Database
warnings.filterwarnings('ignore')

In [4]:
query_produto = """
SELECT 
pdt.codprod,
pdt.codsec,
pdt.codepto,
pdt.descricao,
pdt.qtunitcx,
pdt.unidade
FROM pcprodut pdt
"""
dim_produto = pd.read_sql_query ( query_produto, connection )

In [1]:
#dim_produto

In [6]:
query_estoque = """
select 
pest.codprod,
pest.qtest,
pest.qtvendmes,
pest.qtvendmes1,
pest.qtvendmes2,
pest.qtvendmes3,
pest.dtultent,
pest.dtultsaida
from pcest pest
where dtultent is not null
"""
dim_estoque = pd.read_sql_query ( query_estoque, connection )

In [2]:
#dim_estoque

In [3]:
#dim_estoque.sort_values(by="QTEST", axis=0, ascending=False).reset_index(drop=True)

In [9]:
query_usuario = """
SELECT
pcu.codusur,
pcu.nome,
pcu.codsupervisor,
pcu.bloqueio,
pcu.dtinicio,
pcu.dttermino,
pcu.dtultvenda 
FROM pcusuari pcu
"""
dim_usuario = pd.read_sql_query ( query_usuario, connection )

In [4]:
#dim_usuario

In [11]:
query_departamento = """
SELECT 
pdp.codepto,
pdp.descricao
FROM pcdepto pdp
"""
dim_departamento = pd.read_sql_query ( query_departamento, connection )

In [5]:
#dim_departamento

In [13]:
query_secao = """
SELECT 
psc.codsec,
psc.descricao,
psc.codepto
FROM pcsecao psc
"""
dim_secao = pd.read_sql_query ( query_secao, connection )

In [6]:
#dim_secao

In [15]:
query_plano_pagamento = """
select 
ppg.codplpag,
ppg.descricao
from pcplpag ppg
"""
dim_plano_pagamento = pd.read_sql_query (query_plano_pagamento, connection )

In [7]:
#dim_plano_pagamento

In [17]:
query_supervisao = """
SELECT CODSUPERVISOR, NOME FROM PCSUPERV
"""
dim_supervisao = pd.read_sql_query ( query_supervisao, connection )

In [8]:
#dim_supervisao

In [19]:
query_clientes = """
SELECT 
pcl.codcli,
pcl.codusur1,
pcl.codusur2,
pcl.cliente,
pcl.fantasia,
pcl.bairroent,
pcl.codcidade,
pcl.dtcadastro,
pcl.dtprimcompra,
pcl.dtultcomp,
pcl.limcred,
pcl.bloqueio,
pcl.obs
FROM PCCLIENT pcl
WHERE pcl.dtexclusao is null
"""
dim_clientes = pd.read_sql_query ( query_clientes, connection )

In [9]:
#dim_clientes

In [22]:
# Transform NA for 0
dim_clientes['CODUSUR2'] = dim_clientes['CODUSUR2'].fillna(0)

# Transform data type to float for integer
dim_clientes['CODUSUR2'] = dim_clientes['CODUSUR2'].astype(int)

# Data Extraction PCPEDI by Semester 

In [45]:
base_pcpedc_2022 = """
SELECT
pc.numped, 
pc.data, 
pc.dtfat,
pc.dtentrega, 
pc.vlcustofin, 
pc.vltabela, 
pc.vltotal,
pc.codcli, 
pc.codusur,
pcu.codsupervisor,
pcl.codcidade, 
pc.tipovenda,
pc.codcob,
pc.codplpag,
pc.condvenda, 
pc.totpeso, 
pc.totvolume, 
pc.numitens
FROM pcpedc pc INNER JOIN pcclient pcl  ON ( pc.codcli = pcl.codcli)
               INNER JOIN pcusuari pcu  ON ( pcl.codusur1 = pcu.codusur) 
               INNER JOIN pcsuperv psp  ON ( pcu.codsupervisor = psp.codsupervisor)
               INNER JOIN pcplpag ppg   ON ( pc.codplpag = ppg.codplpag)
                LEFT JOIN pccidade pcd  ON ( pcl.codcidade = pcd.codcidade )       
WHERE pc.data between '01-jan-2022' AND '30-abr-2022' 
AND pc.posicao = 'F'
"""
base_pcpedc_2022 = pd.read_sql_query ( base_pcpedc_2022 , connection )

In [51]:
base_pcpedc_2022

,NUMPED,DATA,DTFAT,DTENTREGA,VLCUSTOFIN,VLTABELA,VLTOTAL,CODCLI,CODUSUR,CODSUPERVISOR,CODCIDADE,TIPOVENDA,CODCOB,CODPLPAG,CONDVENDA,TOTPESO,TOTVOLUME,NUMITENS
0,306005297,2022-04-11,2022-04-12,2022-04-13,1171.59,1555.68,1555.68,8378,306,138,16418,VP,BK,4,1,80.000,0.192000,8
1,306005215,2022-03-14,2022-03-14,2022-03-15,1108.04,1458.45,1458.45,8378,306,138,16418,VP,BK,4,1,75.000,0.180000,5
2,306005076,2022-01-25,2022-01-26,2022-01-27,1747.65,2222.40,2222.40,8378,306,138,16418,VP,BK,4,1,120.000,0.288000,8
3,306004985,2022-01-03,2022-01-03,2022-01-04,1129.94,1481.60,1481.60,8378,306,138,16418,VP,BK,4,1,80.000,0.192000,8
4,2022000154,2022-03-28,2022-03-28,2022-03-29,156.23,150.96,162.00,17395,2022,61,16418,VV,DH,1,1,14.100,0.060000,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4804,2022000017,2022-02-11,2022-02-14,2022-02-15,547.69,708.47,708.47,18605,2022,61,16418,VP,BK,3,1,140.000,0.351000,2
4805,321007808,2022-04-20,2022-04-30,2022-05-04,371.56,480.83,508.56,18631,321,136,16339,VV,DH,1,1,25.845,0.089907,12
4806,321007696,2022-03-21,2022-03-25,2022-03-30,221.15,281.70,293.34,18631,321,136,16339,VP,BK,10,1,23.896,0.114976,8
4807,403000024,2022-04-12,2022-04-22,2022-04-27,588.78,733.71,733.71,18657,403,136,16321,VV,DH,1,1,20.000,0.000000,1


In [54]:
venda_vista = base_pcpedc_2022[base_pcpedc_2022['TIPOVENDA']=='VV']
venda_vista

,NUMPED,DATA,DTFAT,DTENTREGA,VLCUSTOFIN,VLTABELA,VLTOTAL,CODCLI,CODUSUR,CODSUPERVISOR,CODCIDADE,TIPOVENDA,CODCOB,CODPLPAG,CONDVENDA,TOTPESO,TOTVOLUME,NUMITENS
4,2022000154,2022-03-28,2022-03-28,2022-03-29,156.23,150.96,162.00,17395,2022,61,16418,VV,DH,1,1,14.100,0.060000,3
21,352005307,2022-03-17,2022-03-17,2022-03-18,91.10,115.45,115.45,17360,352,138,16418,VV,BNFT,8,11,6.250,0.015000,4
23,352005195,2022-02-23,2022-02-24,2022-02-25,118.98,125.81,125.81,17360,352,138,16418,VV,BNFT,8,11,11.750,0.050000,2
24,352005046,2022-01-27,2022-01-28,2022-01-31,363.54,446.98,446.98,17360,352,138,16418,VV,BNFT,8,11,48.000,0.184800,1
25,352005045,2022-01-27,2022-01-28,2022-01-31,105.83,128.46,128.46,17360,352,138,16418,VV,BNFT,8,11,14.650,0.026409,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4801,300006491,2022-02-23,2022-02-26,2022-03-02,955.65,1172.79,1172.79,18492,300,138,16310,VV,DH,1,1,112.732,0.432616,21
4802,300006381,2022-01-25,2022-01-27,2022-02-01,106.94,136.32,136.32,18492,300,138,16310,VV,DH,1,1,3.240,0.096768,3
4803,300006380,2022-01-25,2022-01-27,2022-02-01,1634.41,2077.35,2097.24,18492,300,138,16310,VV,DH,1,1,145.895,0.497234,51
4805,321007808,2022-04-20,2022-04-30,2022-05-04,371.56,480.83,508.56,18631,321,136,16339,VV,DH,1,1,25.845,0.089907,12


In [55]:
venda_prazo = base_pcpedc_2022[base_pcpedc_2022['TIPOVENDA']=='VP']
venda_prazo

,NUMPED,DATA,DTFAT,DTENTREGA,VLCUSTOFIN,VLTABELA,VLTOTAL,CODCLI,CODUSUR,CODSUPERVISOR,CODCIDADE,TIPOVENDA,CODCOB,CODPLPAG,CONDVENDA,TOTPESO,TOTVOLUME,NUMITENS
0,306005297,2022-04-11,2022-04-12,2022-04-13,1171.59,1555.68,1555.68,8378,306,138,16418,VP,BK,4,1,80.000,0.192000,8
1,306005215,2022-03-14,2022-03-14,2022-03-15,1108.04,1458.45,1458.45,8378,306,138,16418,VP,BK,4,1,75.000,0.180000,5
2,306005076,2022-01-25,2022-01-26,2022-01-27,1747.65,2222.40,2222.40,8378,306,138,16418,VP,BK,4,1,120.000,0.288000,8
3,306004985,2022-01-03,2022-01-03,2022-01-04,1129.94,1481.60,1481.60,8378,306,138,16418,VP,BK,4,1,80.000,0.192000,8
5,306005368,2022-04-27,2022-04-27,2022-04-28,1260.60,1514.48,1542.86,8251,306,138,16418,VP,BK,31,1,94.600,0.387000,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4799,300006717,2022-04-26,2022-04-30,2022-05-03,789.74,968.06,978.38,18492,300,138,16310,VP,BK,29,1,97.464,0.363509,21
4800,300006616,2022-03-29,2022-03-31,2022-04-05,1021.03,1233.82,1235.88,18492,300,138,16310,VP,BK,10,1,108.820,0.327543,19
4804,2022000017,2022-02-11,2022-02-14,2022-02-15,547.69,708.47,708.47,18605,2022,61,16418,VP,BK,3,1,140.000,0.351000,2
4806,321007696,2022-03-21,2022-03-25,2022-03-30,221.15,281.70,293.34,18631,321,136,16339,VP,BK,10,1,23.896,0.114976,8


# Create DW SQlite

In [59]:
# Create table in database sqlite

#Create database SQlite
db = sa.create_engine ('sqlite:///base_tcc.sqlite', echo=False)
conn = db.connect()

In [60]:
query = """
SELECT name
FROM sqlite_master
WHERE type = 'table'
"""
table = pd.read_sql_query ( query, conn )
table

,name


In [31]:
#create table

query_table_products = """
CREATE TABLE products(
CODPROD     INTEGER,
DESCRICAO   TEXT,
CODFORNEC   INTEGER,
CODEPTO     INTEGER,
CODSEC      INTEGER,
QTUNITCX    REAL    
)
"""

#só a primeira vez
conn.execute(query_table_products)


#insert data to dataframe from table in sqlite database
dim_produto.to_sql('products', con=conn, if_exists='replace', index=False)

8425

In [32]:
#create table

query_table_inventory = """
CREATE TABLE inventory(
CODPROD      INTEGER, 
QTEST        REAL, 
DTULTENT     TEXT, 
DTULTSAIDA   TEXT, 
QTVENDMES    REAL, 
QTVENDMES1   REAL, 
QTVENDMES2   REAL, 
QTVENDMES3   REAL, 
QTVENDDIA    REAL    
)
"""

#só a primeira vez
conn.execute(query_table_inventory)


#insert data to dataframe from table in sqlite database
dim_estoque.to_sql('inventory', con=conn, if_exists='replace', index=False)

7876

In [33]:
#create table

query_table_users = """
CREATE TABLE users(
CODUSUR          INTEGER, 
NOME             TEXT,  
CODSUPERVISOR    INTEGER, 
BLOQUEIO         TEXT, 
DTINICIO         TEXT, 
DTTERMINO        TEXT, 
DTULTVENDA       TEXT
)
"""

#só a primeira vez
conn.execute(query_table_users)


#insert data to dataframe from table in sqlite database
dim_usuario.to_sql('users', con=conn, if_exists='replace', index=False)

267

In [34]:
#create table

query_table_departments = """
CREATE TABLE departments(
CODEPTO          INTEGER, 
DESCRICAO        TEXT,  
CODSUPERVISOR    TEXT
)
"""

#só a primeira vez
conn.execute(query_table_departments)


#insert data to dataframe from table in sqlite database
dim_departamento.to_sql('departments', con=conn, if_exists='replace', index=False)

61

In [35]:
#create table

query_table_secao = """
CREATE TABLE section(
CODSEC       INTEGER, 
DESCRICAO    TEXT, 
CODEPTO      INTEGER
)
"""

#só a primeira vez
conn.execute(query_table_secao)


#insert data to dataframe from table in sqlite database
dim_secao.to_sql('section', con=conn, if_exists='replace', index=False)

245

In [29]:
#create table

query_table_pagamento = """
CREATE TABLE plano_pag(
CODPLPAG     INTEGER,
DESCRICAO    TEXT
)
"""

#só a primeira vez
conn.execute(query_table_pagamento)


#insert data to dataframe from table in sqlite database
dim_plano_pagamento.to_sql('plano_pag', con=conn, if_exists='replace', index=False)

31

In [36]:
#create table

query_table_supervisao = """
CREATE TABLE supervision(
CODSUPERVISOR       INTEGER, 
NOME                TEXT, 
POSICAO             INTEGER
)
"""

#só a primeira vez
conn.execute(query_table_supervisao)


#insert data to dataframe from table in sqlite database
dim_supervisao.to_sql('supervision', con=conn, if_exists='replace', index=False)

23

In [37]:
#create table

query_table_clientes = """
CREATE TABLE clients(
CODCLI       INTEGER, 
CLIENTE      TEXT, 
FANTASIA     TEXT, 
BAIRROENT    TEXT, 
MUNICENT     TEXT, 
CODUSUR1     INTEGER, 
CODUSUR2     INTEGER, 
BLOQUEIO     TEXT, 
DTULTCOMP
)
"""

#só a primeira vez
conn.execute(query_table_clientes)


#insert data to dataframe from table in sqlite database
dim_clientes.to_sql('clients', con=conn, if_exists='replace', index=False)

6427

In [39]:
dim_clientes.dtypes

CODCLI                   int64
CODUSUR1                 int64
CODUSUR2                 int32
CLIENTE                 object
FANTASIA                object
BAIRROENT               object
CODCIDADE              float64
DTCADASTRO      datetime64[ns]
DTPRIMCOMPRA    datetime64[ns]
DTULTCOMP       datetime64[ns]
LIMCRED                float64
BLOQUEIO                object
OBS                     object
dtype: object

In [10]:
#dim_produto

In [40]:
#create table

query_table_base_pcpedc = """
CREATE TABLE base_pcpedc(
numped, 
data, 
dtfat,
dtentrega, 
vlcustofin, 
vltabela, 
vltotal,
codcli, 
codusur,
codsupervisor,
codcidade, 
tipovenda,
codcob,
codplpag,
condvenda, 
totpeso, 
totvolume, 
numitens
)
"""

#só a primeira vez
conn.execute(query_table_base_pcpedc)


#insert data to dataframe from table in sqlite database
base.to_sql('base_pcpedc', con=conn, if_exists='replace', index=False)

643477